In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import sys
import pandas as pd
import numpy as np

import os
import warnings
warnings.filterwarnings(action='ignore')


In [7]:
### 사용자 정의 함수
%cd /content/drive/MyDrive/project3/

# Add the paths to the modules to the Python path

sys.path.append(os.path.abspath('./code/image_function'))

from image_load import resize_image, load_image, load_and_resize_images, convert_to_threshold
from vram_reset import reset_vram

# reset_vram() - > vram 초기화

/content/drive/.shortcut-targets-by-id/1gdU-KuXDUU7v2o8X7TdV0JgGiz603mCc/project3


In [8]:
food_path = '/content/drive/MyDrive/project3/data/traindata/raw_image/구이/갈비구이'

In [9]:
def make_anchor(anchor_size, anchor_aspect_ratio) :

    anchors = [] # [x,y,w,h]로 이루어진 리스트
    anchors_state = [] # 이 앵커를 훈련에 쓸건가? 각 앵커별로 사용 여부를 나타냅니다.

    # 앵커 중심좌표 간격
    interval_x = 16
    interval_y = 16

    # 2단 while문 생성
    x = 8
    y = 8
    while(y <= 224): # 8~208 = 14개
        while(x <= 224): # 8~208 = 14개
            # k개의 앵커 생성. 여기서 k = len(anchor_size) * len(anchor_aspect_ratio)다
            for i in range(0, len(anchor_size)) :
                for j in range(0, len(anchor_aspect_ratio)) :
                    anchor_width = anchor_aspect_ratio[j][0] * anchor_size[i]
                    anchor_height = anchor_aspect_ratio[j][1] * anchor_size[i]

                    anchor = [x, y, anchor_width, anchor_height]
                    anchors.append(anchor)
                    anchors_state.append(1)

            x = x + interval_x
        y = y + interval_y
        x = 8
    return anchors, anchors_state

anchor_size = [128, 256, 512] # 논문에서 PASCAL VOC 2007 데이터셋을 사용할 때 지정한 앵커 사이즈를 그대로 사용합니다.
anchor_aspect_ratio = [[1,1],[1,0.5], [0.5,1]] # W*L기준 ratio 리스트입니다.
anchors, anchors_state = make_anchor(anchor_size, anchor_aspect_ratio)

In [10]:
def align_anchor(anchors_for_Image, anchors_state, Ground_Truth_Box_list):

    # 각 앵커는 해당 위치에서 구한 여러가지 Ground truth Box와의 ioU 중 제일 높은거만 가져옵니다.
    IoU_List = []
    Ground_truth_box_Highest_IoU_List = [] # 각 앵커가 어떤 Ground Truth Box를 보고 가장 높은 IoU를 계산했는지 저장합니다.

    for i in range(0, len(anchors_for_Image)):

        IoU_max = 0
        ground_truth_box_Highest_IoU = [0,0,0,0]

        anchor_for_thisImage = anchors_for_Image[i]

        anchor_minX = anchor_for_thisImage[0] - (anchor_for_thisImage[2]/2)
        anchor_minY = anchor_for_thisImage[1] - (anchor_for_thisImage[3]/2)
        anchor_maxX = anchor_for_thisImage[0] + (anchor_for_thisImage[2]/2)
        anchor_maxY = anchor_for_thisImage[1] + (anchor_for_thisImage[3]/2)

        anchor = [anchor_minX, anchor_minY, anchor_maxX, anchor_maxY]

        for j in range(0, len(Ground_Truth_Box_list)):

            ground_truth_box = Ground_Truth_Box_list[j]

            InterSection_min_x = max(anchor[0], ground_truth_box[0])
            InterSection_min_y = max(anchor[1], ground_truth_box[1])

            InterSection_max_x = min(anchor[2], ground_truth_box[2])
            InterSection_max_y = min(anchor[3], ground_truth_box[3])

            InterSection_Area = 0

            if (InterSection_max_x - InterSection_min_x + 1) >= 0 and (InterSection_max_y - InterSection_min_y + 1) >= 0 :
                InterSection_Area = (InterSection_max_x - InterSection_min_x + 1) * (InterSection_max_y - InterSection_min_y + 1)

            box1_area = (anchor[2] - anchor[0]) * (anchor[3] - anchor[1])
            box2_area = (ground_truth_box[2] - ground_truth_box[0]) * (ground_truth_box[3] - ground_truth_box[1])
            Union_Area = box1_area + box2_area - InterSection_Area

            IoU = (InterSection_Area/Union_Area)
            if IoU > IoU_max :
                IoU_max = IoU
                ground_truth_box_Highest_IoU = ground_truth_box

        IoU_List.append(IoU_max)
        Ground_truth_box_Highest_IoU_List.append(ground_truth_box_Highest_IoU)

        # 한 위치에 9개의 앵커 존재 -> 9개 앵커에 대한 IoU를 계산할 때마다 모아서 Positive, Negative 앵커 분류
        if i % 9 == 8 :
            IoU_List_inOneSpot = IoU_List[i-8:i+1]
            for num in list(range(i-8, i + 1)):
                if IoU_List[num] > 0.7 or (max(IoU_List_inOneSpot) == IoU_List[num] and IoU_List[num] >= 0.3): # positive anchor
                    anchors_state[num] = 2
                elif IoU_List[num] < 0.3 : # negative anchor
                    anchors_state[num] = 1
                else: # 애매한 앵커들
                    anchors_state[num] = 0

    Ground_truth_box_Highest_IoU_List = np.asarray(Ground_truth_box_Highest_IoU_List)
    Ground_truth_box_Highest_IoU_List = np.reshape(Ground_truth_box_Highest_IoU_List, (-1, 4))

    return anchors_state, Ground_truth_box_Highest_IoU_List # 각 앵커의 상태, (모든)앵커가 IoU 계산에 참조한 Ground Truth Box